In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('missing').getOrCreate()

In [2]:
training=spark.read.csv('test5.csv',header=True,inferSchema=True)
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [3]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [4]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [5]:
#[Age,Experience]---->new feature---->independent feature

In [7]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['age','Experience'],outputCol="Independent Features")

In [8]:
output=featureassembler.transform(training)

In [9]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [10]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [12]:
finalized_data=output.select('Independent Features','Salary')

In [13]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [16]:
from pyspark.ml.regression import LinearRegression
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features',labelCol='Salary')
regressor=regressor.fit(train_data)

In [17]:
#coefficient
regressor.coefficients

DenseVector([-102.53, 1688.6818])

In [18]:
regressor.intercept

16470.03994673731

In [19]:
pred_result=regressor.evaluate(test_data)

In [20]:
pred_result.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [30.0,8.0]| 25000|26903.59520639148|
+--------------------+------+-----------------+



In [22]:
pred_result.meanAbsoluteError,pred_result.meanSquaredError

(1903.595206391481, 3623674.709796625)